# Trabalhando dados ENEM 2020

In [2]:
import zipfile
import requests
from io import BytesIO
import os

In [6]:
%%time

url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"

os.makedirs("./enem2020", exist_ok=True)
filebytes = BytesIO(requests.get(url,stream=True).content)

myzip= zipfile.ZipFile(filebytes)
myzip.extractall("./enem2020")

CPU times: user 28.5 s, sys: 11.2 s, total: 39.7 s
Wall time: 4min 5s


# Trabalhando com PySpark

In [7]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 78 kB/s s eta 0:00:01    |███▊                            | 32.3 MB 6.0 MB/s eta 0:00:42                        | 43.4 MB 27.1 MB/s eta 0:00:09     |██████████▊                     | 94.6 MB 20.7 MB/s eta 0:00:10     |███████████▍                    | 99.8 MB 9.2 MB/s eta 0:00:20     |██████████████▎                 | 125.3 MB 4.0 MB/s eta 0:00:40     |██████████████▊                 | 129.6 MB 4.0 MB/s eta 0:00:39     |██████████████▊                 | 129.6 MB 4.0 MB/s eta 0:00:39     |███████████████                 | 131.7 MB 4.0 MB/s eta 0:00:38     |███████████████                 | 132.3 MB 4.0 MB/s eta 0:00:38.2 MB 850 kB/s eta 0:02:51| 142.2 MB 850 kB/s eta 0:02:44██▌               | 145.0 MB 12.4 MB/s eta 0:00:117 MB 13.0 MB/s eta 0:00:08     |█████████████████████▍          | 187.8 MB 799 kB/s eta 0:01:58     |██████████████████████          | 192.6 MB 799 kB/s eta 0:01:51     |███████████████████████████▎    | 240.1 MB 14.0 M

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [9]:
spark = SparkSession.builder.appName("ENEMTeste").getOrCreate()

22/03/27 21:24:14 WARN Utils: Your hostname, MacBook-de-Michelle.local resolves to a loopback address: 127.0.0.1; using 10.0.1.7 instead (on interface en0)
22/03/27 21:24:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/27 21:24:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# leitura dos dados

enem = (
spark.
    read
    .format('csv')
    .options(header=True,sep=';',encoding='latin1',inferSchema=True)
    .load("enem2020/DADOS/MICRODADOS_ENEM_2020.csv")
    
)

In [11]:
enem.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [12]:
enem.count()

5783109

In [13]:
enem.groupBy('TP_SEXO').count().show()

+-------+-------+
|TP_SEXO|  count|
+-------+-------+
|      F|3468805|
|      M|2314304|
+-------+-------+



In [15]:
%%time

(
    enem
    .groupBy('TP_SEXO','NO_MUNICIPIO_ESC')
    .agg(
        f.round(f.mean('NU_NOTA_MT').alias("med_mat")),
        f.mean("NU_NOTA_CH").alias("med_ch")
    )
    .show(truncate=False)

)

+-------+-----------------------+------------------------------------+------------------+
|TP_SEXO|NO_MUNICIPIO_ESC       |round(avg(NU_NOTA_MT) AS med_mat, 0)|med_ch            |
+-------+-----------------------+------------------------------------+------------------+
|M      |Santana de Parnaíba    |582.0                               |538.4227272727272 |
|M      |Ponte Nova             |615.0                               |560.0272727272727 |
|M      |Lajedo                 |554.0                               |507.2876543209876 |
|F      |Taquaritinga           |542.0                               |519.6766990291262 |
|M      |São Gonçalo do Amarante|527.0                               |491.9927953890491 |
|M      |Sacramento             |632.0                               |585.35            |
|F      |Ubaitaba               |488.0                               |519.045           |
|F      |Caldas Novas           |497.0                               |493.00059171597627|
|F      |D

In [16]:
enem.createOrReplaceTempView('enem')

22/03/27 21:26:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [17]:
spark.sql("""

SELECT TP_SEXO,NO_MUNICIPIO_ESC,AVG(NU_NOTA_MT) as med_mat,AVG(NU_NOTA_CH) as med_ch from enem GROUP BY TP_SEXO,NO_MUNICIPIO_ESC 

""").show(truncate=False)

+-------+-----------------------+------------------+------------------+
|TP_SEXO|NO_MUNICIPIO_ESC       |med_mat           |med_ch            |
+-------+-----------------------+------------------+------------------+
|M      |Santana de Parnaíba    |582.4966183574879 |538.4227272727272 |
|M      |Ponte Nova             |615.2194805194805 |560.0272727272727 |
|M      |Lajedo                 |553.8636363636363 |507.2876543209876 |
|F      |Taquaritinga           |541.678947368421  |519.6766990291262 |
|M      |São Gonçalo do Amarante|527.3186119873815 |491.9927953890491 |
|M      |Sacramento             |631.8176470588236 |585.35            |
|F      |Ubaitaba               |488.1578947368422 |519.045           |
|F      |Caldas Novas           |496.7033112582781 |493.00059171597627|
|F      |Diamante               |449.52142857142866|420.63571428571424|
|M      |Suzano                 |590.2323119777159 |542.0090909090908 |
|F      |Barreiros              |477.1842105263158 |466.71000000

In [18]:
(
    enem
    .where("TP_ESTADO_CIVIL = 1 AND NO_MUNICIPIO_ESC = 'Curitiba'")
    .groupBy('TP_SEXO','TP_COR_RACA')
    .agg(
        f.min('NU_NOTA_MT').alias("min_mat"),
        f.max('NU_NOTA_MT').alias("max_mat"),
        f.mean("NU_NOTA_MT").alias("med_mt"),
        f.stddev("NU_NOTA_MT").alias("stddev_mt")
    )
    .show(truncate=False)

)

+-------+-----------+-------+-------+-----------------+------------------+
|TP_SEXO|TP_COR_RACA|min_mat|max_mat|med_mt           |stddev_mt         |
+-------+-----------+-------+-------+-----------------+------------------+
|F      |3          |331.2  |872.5  |545.1703933747413|108.93319364475533|
|F      |2          |327.5  |862.0  |521.8843373493977|111.72755460944609|
|M      |2          |361.6  |734.3  |569.4217948717949|95.55694793177153 |
|F      |4          |368.7  |885.7  |611.4232558139535|123.38595369997122|
|M      |3          |327.4  |958.8  |582.1167770419427|111.45622471465045|
|M      |4          |386.1  |944.3  |676.640909090909 |126.25251373250444|
|M      |1          |0.0    |975.0  |634.6792881646655|120.30423369189846|
|F      |0          |378.7  |821.1  |550.9270833333334|97.23770991549486 |
|F      |1          |330.3  |959.9  |589.3043097151206|117.40564185103207|
|M      |0          |380.0  |882.8  |610.16           |149.40504446083747|
|F      |5          |467.

In [23]:
enem.head(10) #TP_COR_RACA

[Row(NU_INSCRICAO=200006271946, NU_ANO=2020, TP_FAIXA_ETARIA=11, TP_SEXO='F', TP_ESTADO_CIVIL=1, TP_COR_RACA=2, TP_NACIONALIDADE=1, TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=11, TP_ESCOLA=1, TP_ENSINO=None, IN_TREINEIRO=0, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, CO_MUNICIPIO_PROVA=1501402, NO_MUNICIPIO_PROVA='Belém', CO_UF_PROVA=15, SG_UF_PROVA='PA', TP_PRESENCA_CN=0, TP_PRESENCA_CH=0, TP_PRESENCA_LC=0, TP_PRESENCA_MT=0, CO_PROVA_CN=None, CO_PROVA_CH=None, CO_PROVA_LC=None, CO_PROVA_MT=None, NU_NOTA_CN=None, NU_NOTA_CH=None, NU_NOTA_LC=None, NU_NOTA_MT=None, TX_RESPOSTAS_CN=None, TX_RESPOSTAS_CH=None, TX_RESPOSTAS_LC=None, TX_RESPOSTAS_MT=None, TP_LINGUA=1, TX_GABARITO_CN=None, TX_GABARITO_CH=None, TX_GABARITO_LC=None, TX_GABARITO_MT=None, TP_STATUS_REDACAO=None, NU_NOTA_COMP1=None, NU_NOTA_COMP2=None, NU_NOTA_COMP3=None, NU_NOTA_COMP4=None, NU_NOTA_COMP5=None, NU_NOTA_REDACAO=Non

In [31]:
enem.select("TP_ST_CONCLUSAO").show()

+---------------+
|TP_ST_CONCLUSAO|
+---------------+
|              1|
|              1|
|              2|
|              2|
|              1|
|              1|
|              1|
|              1|
|              1|
|              2|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
+---------------+
only showing top 20 rows



In [34]:
(
    enem
    .where("TP_COR_RACA = '1' OR TP_COR_RACA = '2' AND NO_MUNICIPIO_ESC = 'Belo Horizonte' AND TP_ST_CONCLUSAO = '1'")
    .groupBy('TP_COR_RACA')
    .agg(
        f.min('NU_NOTA_MT').alias("min_mat"),
        f.max('NU_NOTA_MT').alias("max_mat"),
        f.mean("NU_NOTA_MT").alias("med_mt"),
        f.stddev("NU_NOTA_MT").alias("stddev_mt")
    )
    .show(truncate=False)

)

+-----------+-------+-------+-----------------+------------------+
|TP_COR_RACA|min_mat|max_mat|med_mt           |stddev_mt         |
+-----------+-------+-------+-----------------+------------------+
|1          |0.0    |975.0  |557.5818172798417|123.18566850175235|
+-----------+-------+-------+-----------------+------------------+



# Leitura de dados com ddt

In [ ]:
enem = dd.read_csv("enem2020/DADOS/MICRODADOS_ENEM_2020.csv",sep=';',decimal='.',encoding='latin1')

In [ ]:
enem.head(10)

In [ ]:
enem.NU_NOTA_MT.dtype

In [ ]:
enem.groupby(['TP_SEXO','TP_COR_RACA']).agg({
    "NU_NOTA_LC":"mean",
    "NU_NOTA_MT":"mean"
}).compute()